+-------------+------------------+------------------+------------------+
|    \ comment|        1         |        0         |        -1        |
|emotion      |                  |                  |                  |
+-------------+------------------+------------------+------------------+
|      1      |  same polarity   |     friendly     |  irony/sarcasm   |
+-------------+------------------+------------------+------------------+
|      0      |  irony/sarcasm   |  same polarity   |  understatement  |
+-------------+------------------+------------------+------------------+
|     -1      |  irony/sarcasm   |     complaint    |  same polarity   |
+-------------+------------------+------------------+------------------+

In [32]:
from datasets import load_dataset
import pandas as pd
import random
import json

In [34]:
# EMOTICON HANDLING
emoticon_sentiment = load_dataset(
    'csv',
    data_files='https://huggingface.co/datasets/viethq1906/emotion-symbols-sentiment/resolve/main/emoticon_sentiment.csv'
)['train'].to_pandas()

# Implement emoticon sentiment classification as done in models/emotion_sentiment.py
import torch
import torch.nn as nn

emoticon_sentiment = emoticon_sentiment[['emoticon', 'sentiment_score']]
emoticon_sentiment['sentiment_score'] = emoticon_sentiment['sentiment_score'].astype(float)

# Add probability columns
emoticon_sentiment['positive'] = 0.0
emoticon_sentiment['neutral'] = 0.0
emoticon_sentiment['negative'] = 0.0

# Calculate probabilities using softmax
softmax = nn.Softmax(dim=-1)

for idx, row in emoticon_sentiment.iterrows():
    score = row['sentiment_score']
    
    if score > 0:
        pos_strength = abs(score) ** 1.5
        scores = torch.tensor([pos_strength, 0.3 * (1 - abs(score)), 0.0], dtype=torch.float32)
    elif score < 0:
        neg_strength = abs(score) ** 1.5
        scores = torch.tensor([0.0, 0.3 * (1 - abs(score)), neg_strength], dtype=torch.float32)
    else:
        scores = torch.tensor([0.0, 1.0, 0.0], dtype=torch.float32)
    
    probs = softmax(scores).numpy()
    emoticon_sentiment.loc[idx, ['positive', 'neutral', 'negative']] = probs

# Determine dominant sentiment
emoticon_sentiment['Sentiment'] = emoticon_sentiment[['positive', 'neutral', 'negative']].idxmax(axis=1)
emoticon_sentiment = emoticon_sentiment[['emoticon', 'Sentiment']]
emoticon_sentiment = emoticon_sentiment.rename(columns={'emoticon': 'Emoticon'})
# Map to same encoding as emoji: Positive=2, Neutral=1, Negative=0
emoticon_sentiment['Sentiment'] = emoticon_sentiment['Sentiment'].map({'positive': 2, 'neutral': 1, 'negative': 0})

# EMOJI HANDLING
emoji_sentiment = load_dataset(
    'csv',
    data_files='https://huggingface.co/datasets/viethq1906/emotion-symbols-sentiment/resolve/main/esr.csv'
)['train'].to_pandas()

emoji_sentiment = emoji_sentiment[['Emoji', 'Positive', 'Negative', 'Neutral']]
emoji_sentiment['Sentiment'] = emoji_sentiment[['Positive', 'Negative', 'Neutral']].idxmax(axis=1)
emoji_sentiment = emoji_sentiment[['Emoji', 'Sentiment']]
emoji_sentiment['Sentiment'] = emoji_sentiment['Sentiment'].map({'Positive': 2, 'Negative': 0, 'Neutral': 1})

print(emoticon_sentiment.head())
print(emoji_sentiment.head())
print(f'Emoticon sentiment shape: {emoticon_sentiment.shape}')
print(f'Emoji sentiment shape: {emoji_sentiment.shape}')

vsfc = load_dataset('ura-hcmut/UIT-VSFC')

  Emoticon  Sentiment
0       $:          0
1       %)          0
2      %-)          0
3      &-:          0
4       &:          0
  Emoji  Sentiment
0     😂          2
1     ❤          2
2     ♥          2
3     😍          2
4     😭          0
Emoticon sentiment shape: (238, 2)
Emoji sentiment shape: (969, 2)


In [41]:
# DATASET HANDLING
def save_to_json(df, filepath):
    records = []
    for _, row in df.iterrows():
        record = {
            "sentence": row['sentence'],
            "sentiment": int(row['sentiment'])  # Convert to int to ensure proper format
        }
        records.append(record)
    
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(records, f, ensure_ascii=False, indent=4)
    
    print(f"Saved {len(records)} records to {filepath}")

# Handling emotion picking
def select_combine_emoticon(sentiment_type, sentiment):
    candidate_emoticon = emoticon_sentiment[emoticon_sentiment['Sentiment'] == (sentiment_type)]
    emoticon = candidate_emoticon.sample(n=1)['Emoticon'].values[0]
    if sentiment_type == 0:
        label = -1
    elif sentiment_type == 1:
        if sentiment == 1 or sentiment == -1:
            label = 0
        elif sentiment == 0:
            label = 1
    elif sentiment_type == 2:
        if sentiment == 1 or sentiment == 0:
            label = 1
        elif sentiment == -1:
            label = 0
    return emoticon, label

def select_combine_emoji(sentiment_type, sentiment):
    candidate_emoji = emoji_sentiment[emoji_sentiment['Sentiment'] == (sentiment_type)]
    emoji = candidate_emoji.sample(n=1)['Emoji'].values[0]
    if sentiment_type == 0:
        label = -1
    elif sentiment_type == 1:
        if sentiment == 1 or sentiment == -1:
            label = 0
        elif sentiment == 0:
            label = 1
    elif sentiment_type == 2:
        if sentiment == 1 or sentiment == 0:
            label = 1
        elif sentiment == -1:
            label = 0
    return emoji, label

def select_emotion(sentiment):
    emotion_type = random.randint(0, 1)
    sentiment_type = random.randint(0, 2)
    if emotion_type == 0:
        return select_combine_emoticon(sentiment_type, sentiment)
    else:
        return select_combine_emoji(sentiment_type, sentiment)

for split_name in vsfc.keys():
    df = vsfc[split_name].to_pandas()
    df = df.dropna(subset=['text', 'label'])
    df = df.rename(columns={'text': 'sentence', 'label': 'sentiment'})
    df['sentiment'] = df['sentiment'].map({'positive': 1, 'neutral': 0, 'negative': -1})
    subset = df.loc[0:int(0.4*df.shape[0])]
        
    for index, row in subset.iterrows():
        sentence = row.sentence
        sentiment = row.sentiment
        punc_pos = sentence.find('.')
        if punc_pos != -1:
            emotion, label = select_emotion(sentiment)
            sentence = sentence[:punc_pos] + emotion + sentence[punc_pos:]
            df.loc[index, 'sentence'] = sentence
            df.loc[index, 'sentiment'] = label

    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    save_to_json(df, f'../res/{split_name}.json')

Saved 11426 records to ../res/train.json
Saved 1583 records to ../res/validation.json
Saved 1583 records to ../res/validation.json
Saved 3166 records to ../res/test.json
Saved 3166 records to ../res/test.json
